# Discover More Cost-Efficient AI Customer Service Agents

Learn how to use the Data Flywheel Foundational Blueprint to continuously discover and promote more cost-efficient variants of an [agentic customer service agent](https://build.nvidia.com/nvidia/ai-virtual-assistant-for-customer-service). 

The customer service agent in this tutorial uses tool calling to handle common service tasks, such as: 

- Product Q&A
- Order status verification
- Returns processing
- Engaging in small talk

These interactions generate logs and tool-calling data that you can use as both evaluation benchmarks and training data. In this tutorial, you'll use this information to drive the flywheel process, fine-tuning smaller LLMs (such as `meta/llama-3.2-1B-instruct`, `meta/llama-3.2-3B-instruct`, `meta/llama-3.1-8B-instruct`) to match accuracy of the currently deployed model (`meta/llama-3.3-70B-instruct`).

## Interfacing with the Blueprint

The following diagram illustrates how admin tools and applications interact with the Flywheel Blueprint, which orchestrates logging, processing, and model management to enable continuous optimization.

![Arch](./arch.png)

Contents: 

0. [Prerequisites](#0)
1. [Load Sample Data](#1)
2. [Create a Flywheel Job](#2)
3. [Monitor Job Status](#3)

<a id="0"></a>
## Prerequisites

### Setup

Import required libraries and configure pandas display options for better readability in notebook outputs.

In [ ]:
import sys
from pathlib import Path
import requests
import time
from datetime import datetime
import json
import pandas as pd


from IPython.display import display, clear_output
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)        # Width of the display in characters
pd.set_option('display.max_colwidth', None)  # Show full content of each cell

In [ ]:
PARENT_DIR = Path.cwd().parent
DATA_DIR = PARENT_DIR / "data"

sys.path.insert(0, str(PARENT_DIR))

---

### Configurations and Health Checks

In [ ]:
# Flywheel Orchestrator URL
API_BASE_URL = "http://0.0.0.0:8000"

# Workload identifiers
WORKLOAD_ID = "aiva-tool-calls"
CLIENT_ID = "aiva-dfw-tutorial"

# Polling interval (in seconds) for monitoring flywheel job
POLL_INTERVAL = 5 

---

<a id="1"></a>
## Step 1: Load Sample Data

Use the provided sample dataset (`data/aiva-final.jsonl`) to simulate real user logs captured while an agentic customer service agent application is running. Each data point uses the OpenAI `ChatCompletions` request format and contains the following attributes:

- `messages` include a `system` message as well as a `user` query.
- `tools` includes a list of functions and parameters available to the LLM to choose from, as well as their parameters and descriptions.
- `responses` are the response generated by the current model (`meta/llama-3.1-70b-instruct`). This response contains the function name(s) and associated argument(s) in a "tool_calls" dict.

In [ ]:
DATA_PATH = DATA_DIR / "aiva_l1_dataset.jsonl"

!head -n1 {DATA_PATH} | jq

The data points generated by the system in response to user queries are considered **ground truth**. 

Ground truth data points are used to **evaluate** and **customize** more efficient models that can perform similarly to the current model. This customization process is analogous to a student-teacher distillation setup, where synthetic data generated from the teacher model is used to fine-tune a student model.

Next, we'll load the data into Elasticsearch using a helper method `load_data_to_elasticsearch`, making it accessible to the Flywheel Orchestrator.

In [ ]:
from src.scripts.load_test_data import load_data_to_elasticsearch

load_data_to_elasticsearch(file_path=DATA_PATH)

---

<a id="2"></a>
## Step 2: Create a Flywheel job

Initiate a Flywheel job by sending a POST request to the `/jobs` API. This triggers the workflow asynchronously.

In production environments, you can automate this process to run at scheduled intervals, in response to specific events, or on demand.

In [ ]:
response = requests.post(
    f"{API_BASE_URL}/api/jobs",
    json={"workload_id": "aiva_2", "client_id": "3434"}
)

response.raise_for_status()
job_id = response.json()["id"]

print(f"Created job with ID: {job_id}")

---

<a id="3"></a>
## Step 3: Monitor Job Status

Submit a GET request to `/jobs/{job_id}` to retrieve the current status.

In [ ]:
def get_job_status(job_id):
    """Get the current status of a job."""
    response = requests.get(f"{API_BASE_URL}/api/jobs/{job_id}")
    response.raise_for_status()
    return response.json()

In [ ]:
get_job_status(job_id)

To simplify the process, you can define utility functions that:

- Periodically retrieve the job status
- Format the output into a table

This makes it easier to compare and analyze the results.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import time
from datetime import datetime
from IPython.display import clear_output

def format_runtime(seconds):
    """Format runtime in seconds to a human-readable string."""
    if seconds is None:
        return "-"
    minutes, seconds = divmod(seconds, 60)
    if minutes > 0:
        return f"{int(minutes)}m {int(seconds)}s"
    return f"{int(seconds)}s"

def extract_main_score(score_str):
    try:
        first_score = score_str.split(";")[0]
        value = first_score.split(":")[1].strip()
        return float(value)
    except Exception:
        return 0

def create_results_table(job_data):
    """Create a pandas DataFrame from job data."""
    rows = []
    for nim in job_data["nims"]:
        model_name = nim["model_name"]
        for eval in nim["evaluations"]:
            score_str = "; ".join(f"{k}: {v}" for k, v in eval["scores"].items() if k != "function_name_and_args_accuracy")
            main_score = extract_main_score(score_str)
            rows.append({
                "Model": model_name,
                "Eval Type": eval["eval_type"].upper(),
                "Score": main_score,
                "Percent Done": eval["progress"],
                "Runtime": format_runtime(eval["runtime_seconds"]),
                "Status": "Completed" if eval["finished_at"] else "Running",
                "Started": datetime.fromisoformat(eval["started_at"]).strftime("%H:%M:%S"),
                "Finished": datetime.fromisoformat(eval["finished_at"]).strftime("%H:%M:%S") if eval["finished_at"] else "-"
            })
    if not rows:
        return pd.DataFrame(columns=["Model", "Eval Type", "Scores", "Percent Done", "Runtime", "Status", "Started", "Finished"])
    
    df = pd.DataFrame(rows)

    return df.sort_values(["Model", "Eval Type"])

def create_customization_table(job_data):
    """Create a pandas DataFrame from customization data."""
    customizations = []
    for nim in job_data["nims"]:
        model_name = nim["model_name"]
        for custom in nim["customizations"]:
            customizations.append({
                "Model": model_name,
                "Started": datetime.fromisoformat(custom["started_at"]).strftime("%H:%M:%S"),
                "Epochs Completed": custom["epochs_completed"],
                "Steps Completed": custom["steps_completed"],
                "Finished": datetime.fromisoformat(custom["finished_at"]).strftime("%H:%M:%S") if custom["finished_at"] else "-",
                "Status": "Completed" if custom["finished_at"] else "Running",
                "Runtime": format_runtime(custom["runtime_seconds"]),
                "Percent Done": custom["progress"],
            })
   
    if not customizations:
        customizations = pd.DataFrame(columns=["Model", "Started", "Epochs Completed", "Steps Completed", "Finished", "Runtime", "Percent Done"])
    customizations = pd.DataFrame(customizations)
    return customizations.sort_values(["Model"])

def monitor_job(job_id):
    """Monitor a job and display its progress in a table."""
    print(f"Monitoring job {job_id}...")
    print("Press Ctrl+C to stop monitoring")
    
    while True:
        try:
            clear_output(wait=True)

            fig, ax = plt.subplots(figsize=(10, 6))
            job_data = get_job_status(job_id)
            results_df = create_results_table(job_data)
            customizations_df = create_customization_table(job_data)
            clear_output(wait=True)
            print(f"Job Status: {job_data['status']}")
            print(f"Total Records: {job_data['num_records']}")
            print(f"Last Updated: {datetime.now().strftime('%H:%M:%S')}")
            print("\nResults:")
            display(results_df)
            print("\nCustomizations:")
            display(customizations_df)
            display(job_data)

            # Plot 1: Evaluation Scores
            ax.set_title("Evalulation Results", fontsize=14)
            if not results_df.empty:
                pivot_df = results_df.pivot(index="Model", columns="Eval Type", values="Score").fillna(0)
                pivot_df.plot(kind='bar', ax=ax)
                ax.set_ylabel("Eval Metrics")
                ax.set_ylim(0, 1)
                ax.legend(title="Eval Type")
                ax.grid(axis='y', linestyle='--', alpha=0.7)
            else:
                ax.text(0.5, 0.5, "No Evaluation Data", ha='center', va='center')

            plt.tight_layout()
            plt.show()                        
            time.sleep(POLL_INTERVAL)
            
        except KeyboardInterrupt:
            print("\nMonitoring stopped by user")
            break
        except Exception as e:
            print(f"\nError: {str(e)}")
            break

In [ ]:
# Start monitoring the job
monitor_job(job_id)